# Fidelity Financial Advisor AI Agent

This notebook uses the LangChain and LangGraph frameworks to build an AI agent that acts as a Fidelity financial advisor, answering user questions about retirement planning.

In [3]:
# Import necessary libraries
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langgraph.graph import StateGraph, END
import os
from typing import TypedDict, Annotated, Dict, Any

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
# Load the Fidelity webpage into a document
loader = WebBaseLoader("https://www.fidelity.com/retirement-planning/overview")
document = loader.load()

In [5]:
# Initialize embeddings and create the Chroma vector store
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(
    documents=document,
    embedding=embeddings,
    persist_directory="./chroma_db"
)

# Create a retriever
retriever = vectorstore.as_retriever()

In [6]:
# Define the state schema
class State(TypedDict):
    input: str
    output: str

# Initialize the language model
llm = ChatOpenAI(temperature=0)

def process_query(state: Dict[str, Any]) -> Dict[str, Any]:
    # Get relevant context
    context = retriever.invoke(state["input"])
    
    # Generate response using the LLM
    prompt = f"Context: {context}\nQuestion: {state['input']}\nProvide a detailed answer about retirement planning based on the context."
    response = llm.invoke(prompt)
    
    # Return updated state
    return {"input": state["input"], "output": response.content}

# Create the agent graph
workflow = StateGraph(state_schema=State)

# Add the processing node
workflow.add_node("process", process_query)

# Set the entry point
workflow.set_entry_point("process")

# Add the final state
workflow.add_edge("process", END)

In [7]:
# Compile the graph into an executable chain
app = workflow.compile()

# Example usage
question = "What are the key factors to consider when planning for retirement?"
result = app.invoke({"input": question})
print("Answer:", result["output"])

Answer: When planning for retirement, there are several key factors to consider to ensure a secure and comfortable retirement. Here are some important aspects to keep in mind:

1. Setting Clear Goals: Define your retirement goals, including the lifestyle you want to maintain, any travel or hobbies you wish to pursue, and any financial milestones you want to achieve.

2. Savings and Investments: Determine how much you need to save for retirement by considering factors such as your current age, desired retirement age, life expectancy, and expected expenses. Utilize retirement accounts like Roth IRAs, Traditional IRAs, and 401(k) plans to save and invest for retirement.

3. Asset Allocation: Create a diversified investment portfolio that balances risk and return based on your risk tolerance, time horizon, and financial goals. Consider a mix of stocks, bonds, and other assets to help grow your retirement savings.

4. Social Security and Pension Benefits: Understand how Social Security bene

## Document Relevance Verification
Before answering questions, we'll verify if the retrieved document content is relevant to the user's retirement planning question. This helps ensure we provide accurate, contextual responses.

In [8]:
# Create a prompt template for checking document relevance
from langchain.prompts import PromptTemplate

relevance_prompt = PromptTemplate(
    template="""You are a financial advisor answering user question about retirement planning. \n
Here is the retrieved document: \n\n {context} \n\n
Here is the user question: {question} \n
If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question.""",
    input_variables=["context", "question"]
)

# Create the relevance checking chain
relevance_chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | relevance_prompt
    | llm
    | StrOutputParser()
)

In [9]:
def process_query(state: Dict[str, Any]) -> Dict[str, Any]:
    # Get relevant context
    context = retriever.invoke(state["input"])
    
    # Check relevance
    relevance = relevance_chain.invoke({
        "context": context,
        "question": state["input"]
    })
    
    if "yes" in relevance.lower():
        # Generate response using the LLM
        response = llm.invoke(
            f"Context: {context}\nQuestion: {state['input']}\nAs a Fidelity financial advisor, provide a detailed answer about retirement planning based on the context."
        )
        return {"input": state["input"], "output": response.content}
    else:
        return {
            "input": state["input"], 
            "output": "I apologize, but I don't have relevant information to answer your retirement planning question accurately. Please try asking a different question specifically about retirement planning."
        }

In [10]:
# Test cases
test_questions = [
    # Relevant question
    "What are the steps that I should take to determine how much I need to save for retirement?",
    
    # Similar question
    "Can you help me calculate my retirement savings target?",
    
    # Irrelevant question
    "What's the best recipe for chocolate chip cookies?"
]

print("Testing the Financial Advisor AI Agent:\n")
for question in test_questions:
    print(f"\nQuestion: {question}")
    result = app.invoke({"input": question})
    print(f"\nAnswer: {result['output']}\n")
    print("-" * 80)

Testing the Financial Advisor AI Agent:


Question: What are the steps that I should take to determine how much I need to save for retirement?

Answer: To determine how much you need to save for retirement, you should follow these steps:

1. Assess Your Current Financial Situation: Start by evaluating your current income, expenses, assets, and debts. This will give you a clear picture of where you stand financially.

2. Set Retirement Goals: Determine the lifestyle you want to have during retirement. Consider factors such as where you want to live, travel plans, healthcare expenses, and any other financial goals you may have.

3. Estimate Your Retirement Expenses: Calculate your expected expenses during retirement, including housing, healthcare, food, transportation, and leisure activities. Don't forget to account for inflation and potential healthcare costs.

4. Calculate Your Retirement Income: Determine your potential sources of retirement income, such as Social Security, pensions, 

## Loading Fund Information
We'll use PyMuPDF (fitz) to load and process the Fidelity Freedom® 2045 Fund (FFFGX) document. This will allow us to incorporate specific fund information into our AI advisor's knowledge base.

In [11]:
import fitz  # PyMuPDF

def load_fund_document(pdf_path: str) -> str:
    """
    Load and extract text from the Fidelity Freedom 2045 Fund PDF document.
    """
    try:
        # Open the PDF document
        doc = fitz.open(pdf_path)
        
        # Extract text from the first page (fund information page)
        text = doc[0].get_text()
        
        # Close the document
        doc.close()
        
        return text
        
    except Exception as e:
        print(f"Error loading PDF: {e}")
        return ""

# Load the fund document
fund_pdf_path = "fidelity_freedom_2045.pdf" 
fund_info = load_fund_document(fund_pdf_path)

# Create a document for the vector store
from langchain_core.documents import Document

fund_doc = Document(
    page_content=fund_info,
    metadata={
        "source": "Fidelity Freedom 2045 Fund (FFFGX) Information",
        "type": "fund_document"
    }
)

# Add to our existing vector store
vectorstore.add_documents([fund_doc])

['47ce40e6-ed90-44cd-803b-533ae85384d5']

In [20]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
import logging
import PyPDF2

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Initialize the ChatOpenAI model
llm = ChatOpenAI(temperature=0)

try:
    # Load and process the PDF document using PyPDF2
    logger.info("Loading PDF document...")
    with open("fidelity_freedom_2045.pdf", 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        text_content = ""
        for page in pdf_reader.pages:
            text_content += page.extract_text()
    
    logger.info(f"Content preview: {text_content[:200]}")
    
    if not text_content.strip():
        raise ValueError("No text content extracted from PDF")

    # Split the document into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    splits = text_splitter.create_documents([text_content])
    logger.info(f"Created {len(splits)} text chunks")

    if not splits:
        raise ValueError("No text chunks created - PDF might be empty or unreadable")

    # Create vector store
    embeddings = OpenAIEmbeddings(
        model="text-embedding-ada-002",
        chunk_size=1000
    )

    # Create vector store from texts
    texts = [split.page_content for split in splits]
    vectorstore = Chroma.from_texts(
        texts=texts,
        embedding=embeddings,
        persist_directory="chroma_db"
    )
    logger.info("Vector store created successfully")

    # Define the questions to ask about the fund
    fund_questions = [
        "What is the name of this fund?",
        "Who is the fund manager?",
        "What is the calendar year return for 2022 for this fund and S&P 500?",
        "What is the Portfolio Net Assets?",
        "What is the Morningstar rating for this fund? How many funds were used to rate this fund?"
    ]

    # Create a function to format the question with relevant context
    def get_relevant_context(question: str) -> str:
        try:
            # Search for relevant chunks in the vector store
            docs = vectorstore.similarity_search(question, k=2)
            # Combine the content from relevant chunks
            return "\n".join([doc.page_content for doc in docs])
        except Exception as e:
            logger.error(f"Error getting context: {str(e)}")
            return "Error retrieving context"

    def create_fund_prompt(question: str, context: str) -> str:
        return f"""Based on the following fund document information, please answer this question: {question}

Document content:
{context}

If the information is not found in the document, please respond with "Information not found in the document."
Please provide a direct and specific answer based only on the information shown in the document."""

    # Process each question
    print("Fund Document Analysis:\n")
    for question in fund_questions:
        try:
            # Get relevant context for the question
            context = get_relevant_context(question)
            
            if context == "Error retrieving context":
                print(f"Question: {question}")
                print("Answer: Unable to process question due to technical error")
                print("-" * 80 + "\n")
                continue
                
            # Create a message with the question and context
            message = HumanMessage(
                content=create_fund_prompt(question, context)
            )
            
            # Get the response from the LLM
            response = llm.invoke([message])
            
            # Print the Q&A
            print(f"Question: {question}")
            print(f"Answer: {response.content}")
            print("-" * 80 + "\n")
        except Exception as e:
            logger.error(f"Error processing question '{question}': {str(e)}")
            print(f"Question: {question}")
            print(f"Error: {str(e)}")
            print("-" * 80 + "\n")

except Exception as e:
    logger.error(f"Fatal error: {str(e)}")
    raise

INFO:__main__:Loading PDF document...
INFO:__main__:Content preview: QUARTERLY FUND REVIEW  |   AS OF MARCH 31, 2025
Fidelity Freedom® 2045 Fund
Investment ApproachFUND INFORMATION
•Fidelity Freedom® Funds (the Funds) are designed so that the target date referenced in 
INFO:__main__:Created 92 text chunks
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:__main__:Vector store created successfully


Fund Document Analysis:



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Question: What is the name of this fund?
Answer: The name of this fund is Fidelity Freedom Fund.
--------------------------------------------------------------------------------



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Question: What is the Portfolio Net Assets?
Answer: Information not found in the document.
--------------------------------------------------------------------------------



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Question: What is the Morningstar rating for this fund? How many funds were used to rate this fund?
Answer: The Morningstar rating for this fund is not provided in the document. The document only mentions the number of funds in the Morningstar category, which are 190, 180, 156, and 105.
--------------------------------------------------------------------------------



INFO:backoff:Backing off send_request(...) for 0.6s (requests.exceptions.ReadTimeout: HTTPSConnectionPool(host='us.i.posthog.com', port=443): Read timed out. (read timeout=15))
